In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris

In [ ]:
import mlflow
import mlflow.sklearn

In [ ]:
X,y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
learning_rate = ['constant','adaptive']
num_layers = [(10),(10,10),(10,10,10)]

In [ ]:
ep = 0
for a in learning_rate:
    for h in num_layers:
        with mlflow.start_run():
            mlp = MLPClassifier(learning_rate=a, hidden_layer_sizes=h, max_iter=400)
            mlp.fit(X_train, y_train)

            predicted = mlp.predict(X_test)

            print(f"Neural Network score: \n\tLearning rate:{a} \n\t \
                    Hidden Layers: {h}")
            print(f"Accuracy: {accuracy_score(predicted,y_test)}")

            mlflow.log_param("learning_rate", a)
            mlflow.log_param("hidden_layers", h)
            mlflow.log_metric("accuracy", accuracy_score(predicted,y_test),step=ep)
            ep += 1

            mlflow.sklearn.log_model(mlp, "model")

In [ ]:
# mlflow.create_experiment('new_experiment')
mlflow.set_experiment('new')

In [ ]:
l1_ratio = 0.5

In [ ]:
# This logistic regression is supposed to fail in
# the first attempt due to C=0.0. The objective is
# to show how ML Flow will show the results of a bad model

ep = 0
for i in np.arange(0,5,0.5):
    with mlflow.start_run():
        lr = LogisticRegression(C=i)
        lr.fit(X_train, y_train)

        predicted = lr.predict(X_test)

        print(f"Logistic Regression score: \n\tC:{i}")
        print(f"Accuracy: {accuracy_score(predicted,y_test)}")

        mlflow.log_param("C", i)
        mlflow.log_metric("accuracy", accuracy_score(predicted,y_test),step=ep)
        ep += 1

        mlflow.sklearn.log_model(lr, "model")

In [ ]:
mlflow.set_experiment('new2')

In [ ]:
#Trying to use the autolog feature

mlflow.sklearn.autolog()

In [ ]:
# This logistic regression is supposed to fail in
# the first attempt due to C=0.0. The objective is
# to show how ML Flow will show the results of a bad model

ep = 0
for i in np.arange(0,5,0.5):
    with mlflow.start_run():
        lr = LogisticRegression(C=i)
        lr.fit(X_train, y_train)

        predicted = lr.predict(X_test)

        print(f"Logistic Regression score: \n\tC:{i}")
        print(f"Accuracy: {accuracy_score(predicted,y_test)}")

        ep += 1

        mlflow.sklearn.log_model(lr, "model")

In [ ]:
mlflow.sklearn.save_model(lr,'lr_model/')

In [ ]:
lr = mlflow.sklearn.load_model('lr_model/')

In [ ]:
X_test[:2]

In [ ]:
lr.predict(X_test[:2])

### Serving model

mlflow models serve -m lr_model/ -h 0.0.0.0 -p 8001

### Requesting model from terminal

curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["a","b","c","d"],"data":[[]]}' http://0.0.0.0:8001/invocations

### Requesting model in python

In [ ]:
import requests

host = '0.0.0.0'
port = '8001'

url = f'http://{host}:{port}/invocations'

headers = {
    'Content-Type': 'application/json',
}

# test_data is a Pandas dataframe with data for testing the ML model
http_data = pd.DataFrame([X_test[0]],columns=['a','b','c','d']).to_json(orient='split')

r = requests.post(url=url, headers=headers, data=http_data)

print(f'Predictions: {r.text}')